In [12]:
import pandas as pd

all_df = pd.read_csv("../data/all_data.csv")
site_df = pd.read_csv("../data/site_metadata.csv")

In [14]:
temp_df = all_df.merge(site_df, on=["CUSTOMER_NAME", "PLANT_NAME"])

all_df["THRM_EFF"] = temp_df["POWER"] / (temp_df["FUEL_FLOW"] * temp_df["FUEL_LHV"])

In [15]:
all_df

,location_id,datetime,CMP_SPEED,POWER,FUEL_FLOW,CO2,CUSTOMER_NAME,PLANT_NAME,ENGINE_ID,THRM_EFF
0,1,2021-01-01 00:00:00,7864.836031,9276.641203,1.420379,3.880132,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1,0.303571
1,1,2021-01-01 01:00:00,7872.746093,9256.520108,1.416093,3.868423,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1,0.303830
2,1,2021-01-01 02:00:00,7860.669743,9251.712294,1.414444,3.863916,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1,0.304026
3,1,2021-01-01 03:00:00,7866.736116,9288.037729,1.418746,3.875670,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1,0.304294
4,1,2021-01-01 04:00:00,7864.576346,9272.706150,1.414983,3.865390,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1,0.304600
...,...,...,...,...,...,...,...,...,...,...
1866088,213,2021-12-31 20:00:00,7823.869934,10369.894938,1.634105,3.928073,MERCIFUL,PREMIUM-MOTH,ENGINE_4,0.294890
1866089,213,2021-12-31 21:00:00,7817.320593,10353.512247,1.632244,3.923601,MERCIFUL,PREMIUM-MOTH,ENGINE_4,0.294760
1866090,213,2021-12-31 22:00:00,7818.344861,10352.367427,1.629405,3.916775,MERCIFUL,PREMIUM-MOTH,ENGINE_4,0.295241
1866091,213,2021-12-31 23:00:00,7814.145063,10331.304330,1.628703,3.915089,MERCIFUL,PREMIUM-MOTH,ENGINE_4,0.294767


In [4]:
all_df.head()
site_df.head()

,CUSTOMER_NAME,PLANT_NAME,LATITUDE,LONGITUDE,ELEVATION,FUEL_N2_MOL_PCT,FUEL_MW,FUEL_LHV,CO2_FUEL_RATIO
0,SPIFFY,SPIRITUAL-POLECAT,61.170356,42.874767,112.000000,4.445063,16.572225,21514.222365,2.621803
1,NONCHALANT,NIFTY-ROOK,37.554515,49.908217,-29.000000,1.053145,16.166097,21526.470829,2.714870
2,NONCHALANT,PREHISTORIC-PETREL,29.190866,60.491702,1552.426025,10.298848,17.273122,21494.438672,2.461189
3,NONCHALANT,THERAPEUTIC-LIONFISH,13.253365,76.411056,867.591553,13.188814,17.619149,21485.251979,2.381895
4,SOFT,ABORIGINAL-PICULET,-68.632002,66.155301,1253.152832,7.581916,16.947813,21503.417436,2.535736


In [8]:
site_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CUSTOMER_NAME    45 non-null     object 
 1   PLANT_NAME       45 non-null     object 
 2   LATITUDE         45 non-null     float64
 3   LONGITUDE        45 non-null     float64
 4   ELEVATION        45 non-null     float64
 5   FUEL_N2_MOL_PCT  45 non-null     float64
 6   FUEL_MW          45 non-null     float64
 7   FUEL_LHV         45 non-null     float64
 8   CO2_FUEL_RATIO   45 non-null     float64
dtypes: float64(7), object(2)
memory usage: 3.3+ KB


In [11]:
import plotly.express as px
px.set_mapbox_access_token("pk.eyJ1IjoiYmVudGhlY29kZXIiLCJhIjoiY2wwNGNyb2ZzMGU4NjNjbnlzMXkxcmRnZiJ9.Jh6_T1hrlSoIjAD0UQG4HA")
fig = px.scatter_mapbox(site_df, lat="LATITUDE", lon="LONGITUDE", hover_name="CUSTOMER_NAME", hover_data=["CUSTOMER_NAME", "PLANT_NAME", "LATITUDE", "LONGITUDE"], height=1000, zoom=1)

fig.update_layout(
    title="Gas Turbine Locations",
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
)

fig.show()

In [20]:
# group by customer name and plant name in all_df and calculate averages of each column
all_df.groupby(["CUSTOMER_NAME", "PLANT_NAME", "ENGINE_ID"]).mean().reset_index()
all_df.groupby(["CUSTOMER_NAME", "PLANT_NAME", "ENGINE_ID"]).std().reset_index()

,CUSTOMER_NAME,PLANT_NAME,ENGINE_ID,location_id,CMP_SPEED,POWER,FUEL_FLOW,CO2,THRM_EFF
0,BUSY,ANCIENT-WASP,ENGINE_1,0.0,3895.495433,3840.875580,0.292981,0.734105,0.059137
1,BUSY,ANCIENT-WASP,ENGINE_2,0.0,4253.159140,3389.588230,0.271514,0.741710,0.058320
2,BUSY,ANCIENT-WASP,ENGINE_3,0.0,4014.283737,4987.613401,0.391990,0.942269,0.080981
3,BUSY,ANCIENT-WASP,ENGINE_4,0.0,3072.851055,1395.348163,0.095176,0.258389,0.018133
4,BUSY,ANCIENT-WASP,ENGINE_5,0.0,3495.267108,4153.356116,0.292559,0.712214,0.061788
...,...,...,...,...,...,...,...,...,...
208,TOUGH,VORACIOUS-PARROT,ENGINE_1,0.0,3624.643531,3348.213705,0.254994,0.600452,0.048391
209,TOUGH,VORACIOUS-PARROT,ENGINE_2,0.0,3318.555243,2407.182848,0.176239,0.411323,0.035043
210,TOUGH,VORACIOUS-PARROT,ENGINE_3,0.0,4087.459562,3109.208935,0.231212,0.629978,0.049346
211,TOUGH,VORACIOUS-PARROT,ENGINE_4,0.0,4492.038115,4531.430188,0.365466,0.855242,0.069967
